# Indian National Flag Validator

This notebook validates an image of the Indian national flag against Bureau of Indian Standards (BIS) rules:

- **Aspect Ratio:** 3:2 (±1%)
- **Colors:** Saffron (#FF9933), White (#FFFFFF), Green (#138808), Chakra Blue (#000080) ±5% RGB tolerance
- **Stripe Proportions:** Each band is 1/3 of flag height
- **Ashoka Chakra:** Diameter 3/4 of white band, 24 spokes, centered

The output is a detailed JSON report of detected defects.

In [10]:
# Import required libraries
import cv2
import numpy as np
import json
from math import isclose
from PIL import Image
import os

In [11]:
# BIS color definitions (RGB)
BIS_COLORS = {
    "saffron": (255, 153, 51),
    "white": (255, 255, 255),
    "green": (19, 136, 8),
    "chakra_blue": (0, 0, 128)
}

def color_deviation(actual, target):
    diff = np.abs(np.array(actual) - np.array(target))
    deviation = np.mean(diff / 255) * 100
    return deviation

In [12]:
def validate_flag(image_path):
    img = Image.open(image_path).convert("RGB")
    img_np = np.array(img)
    h, w, _ = img_np.shape
    report = {}

    # Aspect ratio check
    aspect_ratio = w / h
    report["aspect_ratio"] = {
        "status": "pass" if abs(aspect_ratio - 1.5) <= 0.015 else "fail",
        "actual": round(aspect_ratio, 3)
    }

    # Stripe proportions
    stripe_height = h / 3
    top_band = img_np[0:int(stripe_height)]
    mid_band = img_np[int(stripe_height):int(2*stripe_height)]
    bottom_band = img_np[int(2*stripe_height):]

    def avg_rgb(band):
        return tuple(np.mean(band.reshape(-1, 3), axis=0)[::-1])  # BGR→RGB

    colors_report = {}
    for name, band in zip(["saffron", "white", "green"],
                          [top_band, mid_band, bottom_band]):
        avg_color = avg_rgb(band)
        dev = color_deviation(avg_color, BIS_COLORS[name])
        colors_report[name] = {
            "status": "pass" if dev <= 5 else "fail",
            "deviation": f"{dev:.1f}%"
        }

    # Detect Chakra in white band
    mid_gray = cv2.cvtColor(mid_band, cv2.COLOR_RGB2GRAY)
    circles = cv2.HoughCircles(mid_gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50,
                               param1=50, param2=30, minRadius=10, maxRadius=int(stripe_height/2))
    chakra_report = {}
    if circles is not None:
        circles = np.uint16(np.around(circles))
        x, y, r = circles[0][0]
        ideal_diameter = 0.75 * stripe_height
        chakra_report["status"] = "pass" if abs((2*r) - ideal_diameter) <= 2 else "fail"
        chakra_report["offset_x"] = f"{x - w/2:.1f}px"
        chakra_report["offset_y"] = f"{y - stripe_height/2:.1f}px"
        colors_report["chakra_blue"] = {
            "status": "pass",
            "deviation": f"{color_deviation(mid_band[y-r:y+r, x-r:x+r].mean(axis=(0,1)), BIS_COLORS['chakra_blue']):.1f}%"
        }
    else:
        chakra_report["status"] = "fail"

    report["colors"] = colors_report
    report["stripe_proportion"] = {
        "status": "pass" if isclose(stripe_height, h/3, rel_tol=0.01) else "fail",
        "top": round(len(top_band)/h, 2),
        "middle": round(len(mid_band)/h, 2),
        "bottom": round(len(bottom_band)/h, 2)
    }
    report["chakra_position"] = chakra_report
    # Spoke count detection (advanced step omitted for brevity)

    return json.dumps(report, indent=2)

## Run the Validator

Provide the path to your flag image (PNG/JPG/SVG, ≤5MB, flat colors only).

In [13]:
# Example usage
image_path = "./test/1.jpg"  # Change to your image file
result_json = validate_flag(image_path)
print(result_json)

{
  "aspect_ratio": {
    "status": "pass",
    "actual": 1.5
  },
  "colors": {
    "saffron": {
      "status": "fail",
      "deviation": "53.2%"
    },
    "white": {
      "status": "pass",
      "deviation": "3.7%"
    },
    "green": {
      "status": "pass",
      "deviation": "3.0%"
    },
    "chakra_blue": {
      "status": "pass",
      "deviation": "54.6%"
    }
  },
  "stripe_proportion": {
    "status": "pass",
    "top": 0.33,
    "middle": 0.33,
    "bottom": 0.33
  },
  "chakra_position": {
    "status": "pass",
    "offset_x": "1.0px",
    "offset_y": "0.0px"
  }
}


In [14]:
test_folder = "./test"
jpg_files = [f for f in os.listdir(test_folder) if f.lower().endswith('.jpg')]

for fname in jpg_files:
    img_path = os.path.join(test_folder, fname)
    print(f"\n--- Validation for: {fname} ---")
    print(validate_flag(img_path))


--- Validation for: 1.jpg ---
{
  "aspect_ratio": {
    "status": "pass",
    "actual": 1.5
  },
  "colors": {
    "saffron": {
      "status": "fail",
      "deviation": "53.2%"
    },
    "white": {
      "status": "pass",
      "deviation": "3.7%"
    },
    "green": {
      "status": "pass",
      "deviation": "3.0%"
    },
    "chakra_blue": {
      "status": "pass",
      "deviation": "54.6%"
    }
  },
  "stripe_proportion": {
    "status": "pass",
    "top": 0.33,
    "middle": 0.33,
    "bottom": 0.33
  },
  "chakra_position": {
    "status": "pass",
    "offset_x": "1.0px",
    "offset_y": "0.0px"
  }
}

--- Validation for: 2.jpg ---
{
  "aspect_ratio": {
    "status": "fail",
    "actual": 1.333
  },
  "colors": {
    "saffron": {
      "status": "fail",
      "deviation": "53.2%"
    },
    "white": {
      "status": "pass",
      "deviation": "4.5%"
    },
    "green": {
      "status": "pass",
      "deviation": "3.0%"
    },
    "chakra_blue": {
      "status": "pass",
